<!-- Copyright 2015 Google Inc. All rights reserved. -->

<!-- Licensed under the Apache License, Version 2.0 (the "License"); -->
<!-- you may not use this file except in compliance with the License. -->
<!-- You may obtain a copy of the License at -->

<!--     http://www.apache.org/licenses/LICENSE-2.0 -->

<!-- Unless required by applicable law or agreed to in writing, software -->
<!-- distributed under the License is distributed on an "AS IS" BASIS, -->
<!-- WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. -->
<!-- See the License for the specific language governing permissions and -->
<!-- limitations under the License. -->

# Getting started with the Google Genomics API

In this notebook we'll cover how to make authenticated requests to the [Google Genomics API](https://cloud.google.com/genomics/reference/rest/).

----

NOTE:

* If you're new to notebooks, or want to check out additional samples, check out the full [list](../) of general notebooks.
* For additional Genomics samples, check out the full [list](./) of Genomics notebooks.

## Setup

### Install Python libraries

We'll be using the [Google Python API client](https://github.com/google/google-api-python-client) for interacting with Genomics API. We can install this library, or any other 3rd-party Python libraries from the [Python Package Index (PyPI)](https://pypi.python.org/pypi) using the `pip` package manager.

There are [50+ Google APIs](http://api-python-client-doc.appspot.com/) that you can work against with the Google Python API Client, but we'll focus on the Genomics API in this notebook.

In [1]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python2.7/dist-packages
Cleaning up...


### Create an Authenticated Client

Next we construct a Python object that we can use it to make requests. 

The following snippet shows how we can authenticate using the service account on the Datalab host.  For more detail about authentication from Python, see [Using OAuth 2.0 for Server to Server Applications](https://developers.google.com/api-client-library/python/auth/service-accounts).

In [2]:
from httplib2 import Http
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()
http = Http()
credentials.authorize(http)


And then we create a client for the Genomics API.

In [3]:
from apiclient.discovery import build
genomics = build('genomics', 'v1', http=http)

### Send a request to the Genomics API

Now that we have a Python client for the Genomics API, we can access a variety of different resources. For details about each available resource, see the python client [API docs here](https://google-api-client-libraries.appspot.com/documentation/genomics/v1/python/latest/index.html).

Using our `genomics` client, we'll demonstrate fetching a Dataset resource by ID (the [1000 Genomes dataset](http://googlegenomics.readthedocs.org/en/latest/use_cases/discover_public_data/1000_genomes.html) in this case).

First, we need to construct a request object.

In [4]:
request = genomics.datasets().get(datasetId='10473108253681171589')

Next, we'll send this request to the Genomics API by calling the `request.execute()` method.

In [5]:
response = request.execute()

You will need enable the Genomics API for your project if you have not done so previously.  Click on [this link](https://console.developers.google.com/flows/enableapi?apiid=genomics) to enable the API in your project.

The response object returned is simply a Python dictionary. Let's take a look at the properties returned in the response.

In [6]:
for entry in response.items():
    print "%s => %s" % entry

projectId => genomics-public-data
id => 10473108253681171589
createTime => 1970-01-01T00:00:00.000Z
name => 1000 Genomes


Success! We can see the name of the specified Dataset and a few other pieces of metadata.

Accessing other Genomics API resources will follow this same set of steps. The full [list of available resources within the API is here](https://google-api-client-libraries.appspot.com/documentation/genomics/v1/python/latest/index.html). Each resource has details about the different verbs that can be applied (e.g., [Dataset methods](https://google-api-client-libraries.appspot.com/documentation/genomics/v1/python/latest/genomics_v1.datasets.html)).

## Access Data

In this portion of the notebook, we implement [this same example](https://github.com/googlegenomics/getting-started-with-the-api/tree/master/python) implemented as a python script.  First let's define a few constants to use within the examples that follow.

In [7]:
dataset_id = '10473108253681171589' # This is the 1000 Genomes dataset ID
sample = 'NA12872'
reference_name = '22'
reference_position = 51003835

### Get read bases for a sample at specific a position

First find the read group set ID for the sample.

In [8]:
request = genomics.readgroupsets().search(
  body={'datasetIds': [dataset_id], 'name': sample},
  fields='readGroupSets(id)')
read_group_sets = request.execute().get('readGroupSets', [])
if len(read_group_sets) != 1:
  raise Exception('Searching for %s didn\'t return '
                  'the right number of read group sets' % sample)

read_group_set_id = read_group_sets[0]['id']

Once we have the read group set ID, lookup the reads at the position in which we are interested.

In [9]:
request = genomics.reads().search(
  body={'readGroupSetIds': [read_group_set_id],
        'referenceName': reference_name,
        'start': reference_position,
        'end': reference_position + 1,
        'pageSize': 1024},
  fields='alignments(alignment,alignedSequence)')
reads = request.execute().get('alignments', [])

And we print out the results.

In [10]:
# Note: This is simplistic - the cigar should be considered for real code
bases = [read['alignedSequence'][
           reference_position - int(read['alignment']['position']['position'])]
         for read in reads]

print '%s bases on %s at %d are' % (sample, reference_name, reference_position)

from collections import Counter
for base, count in Counter(bases).items():
  print '%s: %s' % (base, count)

NA12872 bases on 22 at 51003835 are
C: 1
G: 13


### Get variants for a sample at specific a position

First find the call set ID for the sample.

In [11]:
request = genomics.callsets().search(
  body={'variantSetIds': [dataset_id], 'name': sample},
  fields='callSets(id)')
resp = request.execute()
call_sets = resp.get('callSets', [])
if len(call_sets) != 1:
  raise Exception('Searching for %s didn\'t return '
                  'the right number of call sets' % sample)

call_set_id = call_sets[0]['id']

Once we have the call set ID, lookup the variants that overlap the position in which we are interested.

In [12]:
request = genomics.variants().search(
  body={'callSetIds': [call_set_id],
        'referenceName': reference_name,
        'start': reference_position,
        'end': reference_position + 1},
  fields='variants(names,referenceBases,alternateBases,calls(genotype))')
variant = request.execute().get('variants', [])[0]

And we print out the results.

In [13]:
variant_name = variant['names'][0]
genotype = [variant['referenceBases'] if g == 0
            else variant['alternateBases'][g - 1]
            for g in variant['calls'][0]['genotype']]

print 'the called genotype is %s for %s' % (','.join(genotype), variant_name)

the called genotype is G,G for rs131767
